# **Notebook 1: Data Extraction, Transformation and Loading**

## Objectives

* Import raw data from Kaggle into a dataframe
* Clean data to remove duplicate values and remove outliers
* Identify and handle missing data

## Inputs

* Raw data files from [CO2 Emissions Dataset](https://www.kaggle.com/datasets/shreyanshdangi/co-emissions-across-countries-regions-and-sectors/data)

## Outputs

* Generates clean_data.csv for use in hypothesis testing and visualisations

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Import Packages

Import packages needed to run the notebook

In [7]:
import os
import numpy as np
import pandas as pd

# Import Data

Import raw data into dataframe, ready for processing

In [33]:
# set path to data file
path = "../raw_data/data.csv"

# assign data to dataframe
df_raw = pd.read_csv(path)

# display dataframe
df_raw.head()

,Description,Name,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Country,Afghanistan,1850,AFG,3752993.0,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.436,0.629,NaN,NaN
1,Country,Afghanistan,1851,AFG,3767956.0,NaN,0.0,0.0,NaN,NaN,...,NaN,0.156,0.0,0.0,0.0,0.0,7.500,0.633,NaN,NaN
2,Country,Afghanistan,1852,AFG,3783940.0,NaN,0.0,0.0,NaN,NaN,...,NaN,0.155,0.0,0.0,0.0,0.0,7.560,0.637,NaN,NaN
3,Country,Afghanistan,1853,AFG,3800954.0,NaN,0.0,0.0,NaN,NaN,...,NaN,0.155,0.0,0.0,0.0,0.0,7.620,0.641,NaN,NaN
4,Country,Afghanistan,1854,AFG,3818038.0,NaN,0.0,0.0,NaN,NaN,...,NaN,0.155,0.0,0.0,0.0,0.0,7.678,0.644,NaN,NaN


# Data Cleaning

FIrst I will do some initital data cleaning steps:
1. Check for duplicate rows and remove if found
2. Limit data to country information to fit the requirements of the analysis
3. Limit data to last 50 years to make analysis easier to manage and findings more relevant to current times

In [ ]:
# drop any duplicate rows
df_raw.drop_duplicates(inplace=True)

# select rows where Description is equal to Country
df_raw = df_raw.loc[df_raw['Description'] == "Country"]

# limit data to last 50 years
df_raw = df_raw.loc[df_raw["year"] >= 1975]

df_raw.shape

(9702, 80)

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
